In [1]:
import pandas as pd

ab_data = pd.read_csv('ab_data.csv')

### Data preparation. Предварительная обработка данных ###

In [2]:
ab_data.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [3]:
ab_data.rename(columns={"converted": "isBought"}, inplace=True)
ab_data.group.replace('treatment', 'test', inplace=True)
ab_data.head()

,user_id,timestamp,group,landing_page,isBought
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,test,new_page,0
3,853541,2017-01-08 18:28:03.143765,test,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [4]:
print(ab_data[(ab_data["group"] == "test") & (ab_data["landing_page"] == "old_page")].shape)
print(ab_data[(ab_data["group"] == "test") & (ab_data["landing_page"] == "new_page")].shape)
print(ab_data[(ab_data["group"] == "control") & (ab_data["landing_page"] == "old_page")].shape)
print(ab_data[(ab_data["group"] == "control") & (ab_data["landing_page"] == "new_page")].shape)

(236, 5)
(143622, 5)
(143545, 5)
(218, 5)


In [5]:
ab_data.drop(ab_data[(ab_data["group"] == "test") & (ab_data["landing_page"] == "old_page")].index, inplace=True)
ab_data.drop(ab_data[(ab_data["group"] == "control") & (ab_data["landing_page"] == "new_page")].index, inplace=True)

In [6]:
print(ab_data[(ab_data["group"] == "test") & (ab_data["landing_page"] == "old_page")].shape)
print(ab_data[(ab_data["group"] == "test") & (ab_data["landing_page"] == "new_page")].shape)
print(ab_data[(ab_data["group"] == "control") & (ab_data["landing_page"] == "old_page")].shape)
print(ab_data[(ab_data["group"] == "control") & (ab_data["landing_page"] == "new_page")].shape)

(0, 5)
(143622, 5)
(143545, 5)
(0, 5)


In [7]:
print(ab_data.groupby('group').isBought.mean())

group
control    0.121718
test       0.124312
Name: isBought, dtype: float64


In [8]:
print(ab_data['group'].value_counts())

test       143622
control    143545
Name: group, dtype: int64


### Проведем Z test. Нужно скачать statsmodels ###

In [9]:
!pip install statsmodels

In [10]:
ab_data[ab_data["group"] == "test"].isBought.values

array([0, 0, 1, ..., 1, 1, 1], dtype=int64)

In [11]:
ab_data[ab_data["group"] == "control"].isBought.values

array([0, 0, 1, ..., 1, 1, 1], dtype=int64)

### Импортируем ztest() из statsmodels ###

In [14]:
from statsmodels.stats.weightstats import ztest

# В statsmodels.stats.weightstats.ztest нужно передать две выборки (результаты До/После). 
# Из среднего значения тестовой группы вычиается среднее значение контрольной группы, 
# это делится на объединенную стандартную ошибку

statistic, p_value = ztest(ab_data[ab_data["group"] == "test"].isBought.values,
            ab_data[ab_data["group"] == "control"].isBought.values)

print(f"statistic: {statistic}")
print(f"p_value: {p_value}")

statistic: 2.1164957417289
p_value: 0.03430266698861987


### Тест на пропорции на основе нормального (z)теста ###

In [16]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# Количество успехов для каждой независимой выборки
count = np.array([18, 49]) 

# Количество наблюдений
nobs = np.array([2476, 2663])

stat, pval = proportions_ztest(count, nobs)
print('p_value: {0:0.5f}'.format(pval))


p_value: 0.00044
